In [12]:
from datetime import datetime
import pandas as pd
import feather
import os
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page

In [15]:
#pip uninstall --yes marlinfs

Found existing installation: marlinfs 0.0.1.0rc1
Uninstalling marlinfs-0.0.1.0rc1:
  Successfully uninstalled marlinfs-0.0.1.0rc1
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install marlinfs==0.0.1.9rc1

Note: you may need to restart the kernel to use updated packages.


In [13]:
from marlin.marlin_service_pb2 import DataType, TransformJobType, TransformOutputStores

In [14]:
import marlin

In [15]:
df1 = pd.read_feather('data/demo_data.feather')
df1.head(5)

,cust_id,age,job,marital,education
0,100,56,housemaid,married,basic.4y
1,101,57,services,married,high.school
2,102,37,services,married,high.school
3,103,40,admin.,married,basic.6y
4,104,56,services,married,high.school


In [16]:
event_timestamp = int(datetime.timestamp(datetime(2020,4,30))) # 04/30/2020 @ 12:00am (UTC)
df1['event_timestamp']=event_timestamp

In [17]:
df1.head(1)

,cust_id,age,job,marital,education,event_timestamp
0,100,56,housemaid,married,basic.4y,1588230000


In [18]:
name='transform_1'
version='1'
entities={'cust_id':DataType.LONG}
features={'age': DataType.LONG, 
          'job': DataType.STRING,
          'marital': DataType.STRING, 
          'education': DataType.STRING}
output_stores=[TransformOutputStores.BATCH_STORE, TransformOutputStores.ONLINE_STORE]

In [19]:
client = marlin.client().batch_ingestion_client(name,version,entities,features,output_stores)

In [20]:
@client.transform_function
def process():
    return df1

In [21]:
process()

_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "{"created":"@1606718161.754774000","description":"Error received from peer ipv4:34.213.53.17:443","file":"src/core/lib/surface/call.cc","file_line":1062,"grpc_message":"Socket closed","grpc_status":14}"
>

AssertionError: Required argument `s3_staging_dir` or `work_group` not found.

In [10]:
os.environ["AWS_ATHENA_S3_STAGING_DIR"]="s3://marlin-offline-store/query-results-bucket/"

In [11]:
#os.environ["AWS_ATHENA_S3_STAGING_DIR"]="s3://ingestion-offline-store"

In [12]:
exploration_client = marlin.client().exploration_client()

In [13]:
exploration_client.list_all_transform()

transform_key {
  transform_name: "transform_1"
  version: "1"
}

In [14]:
exploration_client.list_partitions(exploration_client.list_all_transform().transform_key[0])

FileNotFoundError: The specified bucket does not exist

In [15]:
marlin.root_location = 's3://marlin-offline-store'

In [19]:
marlin.MARLIN_CLIENT = MarlinServiceClient(marlin.api_url, marlin.is_secure, marlin.root_location)

In [17]:
from marlin import MarlinServiceClient

In [43]:
exploration_client = marlin.client().exploration_client()

In [44]:
exploration_client.list_all_transform()

transform_key {
  transform_name: "transform_1"
  version: "1"
}

In [45]:
exploration_client.list_partitions(exploration_client.list_all_transform().transform_key[0])

FileNotFoundError: The specified bucket does not exist

In [23]:
marlin.client().batch_store.root_location

's3://marlin-offline-store'

In [37]:
exploration_client.execute("show tables in sampledb")

,tab_name
0,1
1,1602801173_campaign_info
2,1603063839_campaign_info
3,1603063839_demographics
4,1603063839_mktg_model_v1
...,...
20,t
21,t_1
22,transform_1_1
23,x_1


In [36]:
exploration_client.execute("show databases")

,database_name
0,default
1,sampledb


In [41]:
exploration_client.execute("select * from sampledb.transform_1_1")

,ingestion_time,event_time,cust_id,age,education,job,marital,partition_0
0,1606597988962,1588230000,34656,40,high.school,services,divorced,2020-11-28-21
1,1606597988962,1588230000,34655,34,professional.course,technician,single,2020-11-28-21
2,1606597988962,1588230000,34654,27,high.school,services,single,2020-11-28-21
3,1606597988962,1588230000,34653,31,university.degree,services,single,2020-11-28-21
4,1606597988962,1588230000,34652,31,high.school,admin.,single,2020-11-28-21
...,...,...,...,...,...,...,...,...
2994,1606597988961,1588230000,34661,36,university.degree,management,married,2020-11-28-21
2995,1606597988961,1588230000,34660,28,high.school,student,single,2020-11-28-21
2996,1606597988961,1588230000,34659,28,high.school,admin.,married,2020-11-28-21
2997,1606597988962,1588230000,34658,23,high.school,admin.,single,2020-11-28-21


In [33]:
exploration_client.list_all_serving()